## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-18-15-52-27 +0000,bbc,Two Ukrainians working for Russia behind rail ...,https://www.bbc.com/news/articles/c4gknv8nxlzo...
1,2025-11-18-15-51-04 +0000,wapo,Trump HIV prevention plan shuts out South Afri...,https://www.washingtonpost.com/national-securi...
2,2025-11-18-15-50-32 +0000,nyt,House Democrats Press for Vote to Bar Military...,https://www.nytimes.com/2025/11/18/us/politics...
3,2025-11-18-15-49-10 +0000,nyt,We Now Have a Better Picture of China’s Global...,https://www.nytimes.com/2025/11/18/business/ch...
4,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,49
48,epstein,19
277,new,15
358,files,13
20,house,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
265,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...,131
4,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,122
198,2025-11-18-02-00-00 +0000,wsj,President Trump’s grip on the GOP is showing s...,https://www.wsj.com/politics/policy/trumps-gri...,107
201,2025-11-18-01-48-49 +0000,nyt,Trump Has the Power to Release the Epstein Fil...,https://www.nytimes.com/2025/11/17/us/politics...,105
84,2025-11-18-12-38-40 +0000,bbc,When is the Epstein files vote - and why does ...,https://www.bbc.com/news/articles/cq50nvq8znvo...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
265,131,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...
171,60,2025-11-18-06-07-06 +0000,nypost,LA County Sheriff investigating new sexual bat...,https://nypost.com/2025/11/18/us-news/la-count...
66,56,2025-11-18-13-24-46 +0000,cbc,"Fighter jet, investment deals on agenda as Sau...",https://www.cbc.ca/news/world/washington-saudi...
1,41,2025-11-18-15-51-04 +0000,wapo,Trump HIV prevention plan shuts out South Afri...,https://www.washingtonpost.com/national-securi...
282,41,2025-11-17-20-22-42 +0000,nypost,Ex-Disney star ripped for ‘demonic’ app that l...,https://nypost.com/2025/11/17/business/ex-disn...
105,34,2025-11-18-11-33-48 +0000,bbc,Minister insists government committed to asylu...,https://www.bbc.com/news/articles/cvgd2vq4enzo...
244,34,2025-11-17-22-37-45 +0000,nypost,UN Security Council backs Trump’s Gaza cease-fire,https://nypost.com/2025/11/17/world-news/un-se...
260,33,2025-11-17-21-50-00 +0000,wsj,"Market Rout Intensifies, Sweeping Up Everythin...",https://www.wsj.com/finance/stocks/market-rout...
104,33,2025-11-18-11-37-32 +0000,nypost,James Comey trial judge stays order for feds t...,https://nypost.com/2025/11/18/us-news/james-co...
306,32,2025-11-17-18-40-35 +0000,nypost,Bangladesh’s ousted PM Sheikh Hasina sentenced...,https://nypost.com/2025/11/17/world-news/bangl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
